<a href="https://colab.research.google.com/github/wma-spu/welcome/blob/main/DS610_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install java libs and spark.
!apt update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# set environment variables for java and spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
# add pyspark to sys.path
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
dataFrame = spark.read.format("CSV").option("header","true").load('/content/drive/MyDrive/AirlineInfo/*.csv.bz2')
dataFrame.show(5)
dataFrame.printSchema()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    1|         1|        1|   1232|      1225|   1341|      1340|           WN|     2891

In [ ]:
dataFrame = dataFrame.withColumn("ArrDelay",dataFrame.ArrDelay.cast('integer')) \
                     .withColumn("DepDelay",dataFrame.DepDelay.cast('integer')) \
                     .withColumn("Cancelled",dataFrame.Cancelled.cast('integer'))
dataFrame = dataFrame[['UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'DayofMonth', 'DayOfWeek', 'ArrDelay', 'DepDelay', 'Cancelled']]

In [ ]:
dataFrame.show(5)
dataFrame.printSchema()
dataFrameCache = dataFrame.cache()

+-------------+---------+------+----+----------+---------+--------+--------+---------+
|UniqueCarrier|FlightNum|Origin|Dest|DayofMonth|DayOfWeek|ArrDelay|DepDelay|Cancelled|
+-------------+---------+------+----+----------+---------+--------+--------+---------+
|           WN|     2891|   SMF| ONT|         1|        1|       1|       7|        0|
|           WN|      462|   SMF| PDX|         1|        1|       8|      13|        0|
|           WN|     1229|   SMF| PDX|         1|        1|      34|      36|        0|
|           WN|     1355|   SMF| PDX|         1|        1|      26|      30|        0|
|           WN|     2278|   SMF| PDX|         1|        1|      -3|       1|        0|
+-------------+---------+------+----+----------+---------+--------+--------+---------+
only showing top 5 rows

root
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DayofMonth: string (nu

In [35]:
from pyspark.sql.functions import sum
dataFrameCache = dataFrame.groupBy(['UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'DayofMonth', 'DayofWeek']) \
                     .agg(sum('ArrDelay').alias("sum_ArrDelay"), \
                          count('ArrDelay').alias('flight_Count'), \
                          count(lit(when(col('ArrDelay') <= 0, 1))).alias("count_Ontime"), \
                          sum('DepDelay').alias("sum_DepDelay"), \
                          sum('Cancelled').alias("sum_Cancelled")
                          ).cache()

In [39]:
dataFrameCache.show(10)

+-------------+---------+------+----+----------+---------+------------+------------+------------+------------+-------------+
|UniqueCarrier|FlightNum|Origin|Dest|DayofMonth|DayofWeek|sum_ArrDelay|flight_Count|count_Ontime|sum_DepDelay|sum_Cancelled|
+-------------+---------+------+----+----------+---------+------------+------------+------------+------------+-------------+
|           WN|     2083|   SMF| SAN|         1|        1|          57|           2|           1|          69|            0|
|           WN|     2535|   STL| MDW|         1|        1|          -3|           5|           3|          21|            0|
|           WN|      484|   BWI| MDW|         2|        2|         -11|           1|           1|           4|            0|
|           WN|        8|   DAL| AMA|         2|        2|          -8|           3|           2|          -1|            1|
|           WN|     1300|   DEN| PHX|         2|        2|          43|           2|           1|          56|            0|


In [37]:
dataFrameWeekCache = dataFrameCache.groupBy(['UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'DayofWeek']) \
                     .agg(sum('sum_ArrDelay').alias("sum_ArrDelay"), \
                          sum('flight_Count').alias('flight_Count'), \
                          sum('sum_DepDelay').alias("sum_DepDelay"), \
                          sum('sum_Cancelled').alias("sum_Cancelled"), \
                          sum('count_Ontime').alias("count_Ontime")
                          ).cache()
dataFrameWeekCache.show(10)

+-------------+---------+------+----+---------+------------+------------+------------+-------------+------------+
|UniqueCarrier|FlightNum|Origin|Dest|DayofWeek|sum_ArrDelay|flight_Count|sum_DepDelay|sum_Cancelled|count_Ontime|
+-------------+---------+------+----+---------+------------+------------+------------+-------------+------------+
|           9E|     2955|   CID| DTW|        3|         770|          78|         909|            4|          52|
|           9E|     2990|   DTW| CVG|        7|          85|          17|         106|            0|           9|
|           9E|     5654|   PIT| MSP|        7|         -51|          24|          31|            2|          12|
|           9E|     5936|   MEM| MOB|        7|          36|          10|          59|            0|           7|
|           9E|     5961|   TLH| MEM|        6|          98|          19|          90|            0|          10|
|           AA|     1344|   DFW| OMA|        4|         717|          38|         438|  



**1.   Find the # of flights each airline made so far from 1987 until recent.**





In [23]:
countDF = dataFrameWeekCache.groupBy(['UniqueCarrier', 'FlightNum']).sum('flight_Count')
countDF.show(20)

+-------------+---------+-----------------+
|UniqueCarrier|FlightNum|sum(flight_Count)|
+-------------+---------+-----------------+
|           9E|     2955|              673|
|           OH|     5407|             1704|
|           EV|     4829|              952|
|           AA|     1670|             4811|
|           UA|      354|            12959|
|           WN|      563|            16744|
|           CO|     1299|             3260|
|           AS|       59|             1930|
|           DL|     1128|             8954|
|           AS|      548|             2703|
|           UA|     1667|             4255|
|           AA|     1046|             9991|
|           UA|     2060|             2461|
|           NW|     1292|             4536|
|           US|     1878|             2859|
|           DL|     1717|            16589|
|           WN|      116|            10031|
|           9E|     5736|             1123|
|           WN|      179|             7355|
|           UA|     1038|       

**2.   Find the mean departure delay per origination airport.**

In [25]:
delayDF = dataFrameWeekCache.groupBy(['Origin']).agg((sum('sum_DepDelay')/sum('flight_Count')).alias('mean_DepDelay'))
delayDF.show(20)
delayDFCache = delayDF.cache()

+------+------------------+
|Origin|     mean_DepDelay|
+------+------------------+
|   BGM| 4.110733137829912|
|   INL| 6.168539325842697|
|   DLG|10.881406819644006|
|   PSE|1.9634060520760028|
|   MSY| 6.569657564304317|
|   GEG| 6.051149022117038|
|   SNA|  5.82793990516695|
|   BUR| 6.715111222334418|
|   GRB| 4.692966027358289|
|   GTF|3.1398351142995704|
|   IDA|3.0275397097474572|
|   GRR|   5.6374587269899|
|   LWB| 7.633272616879174|
|   PVU|               0.0|
|   EUG| 7.970336079215675|
|   PSG| 13.28594053703837|
|   MYR| 5.598720551147198|
|   PVD|6.8015787129866565|
|   GSO| 7.045275339185953|
|   ISO| 5.764848905870093|
+------+------------------+
only showing top 20 rows



**3. What day the delays are the worst?**

In [26]:
dataFrameWeekCache.createOrReplaceTempView("AIRLINESINFO")
spark.sql("select DayOfWeek, sum(sum_DepDelay)/sum(flight_Count) as avg_delay from AIRLINESINFO " +
          "group by DayOfWeek order by avg_delay desc").show(1)

+---------+------------------+
|DayOfWeek|         avg_delay|
+---------+------------------+
|        5|10.176362510686959|
+---------+------------------+
only showing top 1 row



**4. What is the average departure delay from each airport?**

In [28]:
# This is same as question #2. Find the mean departure delay per origination airport.
delayDFCache.show(20)

+------+------------------+
|Origin|     mean_DepDelay|
+------+------------------+
|   BGM| 4.110733137829912|
|   INL| 6.168539325842697|
|   DLG|10.881406819644006|
|   PSE|1.9634060520760028|
|   MSY| 6.569657564304317|
|   GEG| 6.051149022117038|
|   SNA|  5.82793990516695|
|   BUR| 6.715111222334418|
|   GRB| 4.692966027358289|
|   GTF|3.1398351142995704|
|   IDA|3.0275397097474572|
|   GRR|   5.6374587269899|
|   LWB| 7.633272616879174|
|   PVU|               0.0|
|   EUG| 7.970336079215675|
|   PSG| 13.28594053703837|
|   MYR| 5.598720551147198|
|   PVD|6.8015787129866565|
|   GSO| 7.045275339185953|
|   ISO| 5.764848905870093|
+------+------------------+
only showing top 20 rows



**5. Which day of the week is the most of the flights cancelled?**

In [32]:
#from pyspark.sql.functions import sum, desc
dataFrameWeekCache.groupBy(['DayOfWeek']).agg(sum('sum_Cancelled').alias('sum_Cancelled')).sort(desc('sum_Cancelled')).show(1)

+---------+-------------+
|DayOfWeek|sum_Cancelled|
+---------+-------------+
|        2|       376554|
+---------+-------------+
only showing top 1 row



**6. Which day of the month is the most of the flights cancelled?**

In [33]:
#from pyspark.sql.functions import sum, desc
dataFrameCache.groupBy(['DayofMonth']).agg(sum('sum_Cancelled').alias('sum_cancelled')).sort(desc('sum_cancelled')).show(1)

+----------+-------------+
|DayofMonth|sum_cancelled|
+----------+-------------+
|        11|        99407|
+----------+-------------+
only showing top 1 row



**7. Find the on-time (ArrTime - CRSArrTime <= 0) performance for each unique carrier.**

In [38]:
from pyspark.sql.functions import when, count, col, lit, desc
perfDF = dataFrameWeekCache.groupBy(['UniqueCarrier']).agg((sum('count_Ontime')/sum('flight_Count')).alias('performance')).sort(desc('performance'))
perfDF.show(20)

+-------------+------------------+
|UniqueCarrier|       performance|
+-------------+------------------+
|           HA|0.7494273599184927|
|           AQ|0.6210604130502221|
|           DH|0.6090800627755951|
|           9E|0.6087752820569743|
|           OO| 0.588394965923244|
|           TZ| 0.586800448528448|
|           WN|0.5744281428227538|
|       ML (1)|0.5727513418886269|
|           YV| 0.560270707414659|
|           OH| 0.558461092410828|
|           MQ|0.5579939380234537|
|           XE|0.5532763928309629|
|           B6|0.5479773299779632|
|           NW|0.5479055050560616|
|           FL|0.5449397910536645|
|           AA|0.5380722581960554|
|           EV|0.5255398934893043|
|           CO|0.5241762164458178|
|       PA (1)|0.5189862443648133|
|           F9|0.5189850735570807|
+-------------+------------------+
only showing top 20 rows

